# Merge LLM

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/cg123/mergekit.git
!cd mergekit && pip install -q -e .

Cloning into 'mergekit'...
remote: Enumerating objects: 2304, done.
remote: Counting objects: 100% (619/619), done.
remote: Compressing objects: 100% (286/286), done.
remote: Total 2304 (delta 419), reused 490 (delta 333), pack-reused 1685 (from 1)
Receiving objects: 100% (2304/2304), 709.35 KiB | 4.49 MiB/s, done.
Resolving deltas: 100% (1563/1563), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import yaml

MODEL_NAME = "Marcoro14-7B-slerp"
yaml_config = """
slices:
  - sources:
      - model: SciPhi/SciPhi-Mistral-7B-32k
        layer_range: [0, 32]
      - model: cognitivecomputations/samantha-1.2-mistral-7b
        layer_range: [0, 32]
merge_method: slerp
base_model: SciPhi/SciPhi-Mistral-7B-32k
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16

"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)

In [ ]:
!mergekit-yaml config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle

Streaming output truncated to the last 5000 lines.

pytorch_model-00003-of-00003.bin:  31% 2.78G/9.08G [00:52<01:36, 65.5MB/s]

pytorch_model-00001-of-00003.bin:  28% 2.73G/9.89G [00:52<02:30, 47.6MB/s]


pytorch_model-00002-of-00003.bin:  28% 2.81G/10.0G [00:52<02:01, 59.2MB/s]




pytorch_model-00003-of-00003.bin:  31% 2.80G/9.08G [00:52<01:17, 81.0MB/s]

pytorch_model-00001-of-00003.bin:  28% 2.75G/9.89G [00:52<01:48, 65.5MB/s]


pytorch_model-00002-of-00003.bin:  28% 2.83G/10.0G [00:52<01:39, 72.1MB/s]

pytorch_model-00001-of-00003.bin:  28% 2.76G/9.89G [00:52<01:40, 71.1MB/s]




pytorch_model-00003-of-00003.bin:  31% 2.82G/9.08G [00:52<01:14, 84.0MB/s]


pytorch_model-00002-of-00003.bin:  28% 2.84G/10.0G [00:52<01:38, 72.5MB/s]

pytorch_model-00001-of-00003.bin:  28% 2.77G/9.89G [00:53<01:32, 76.9MB/s]


pytorch_model-00002-of-00003.bin:  29% 2.85G/10.0G [00:53<01:39, 72.0MB/s]

pytorch_model-00001-of-00003.bin:  28% 2.78G/9.89G [00:53<01:36, 73.7MB/s]




pytorch_model-00003-of-

In [ ]:
!pip install -qU huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 6.7 MB/s eta 0:00:00


In [ ]:

from huggingface_hub import ModelCard, ModelCardData
from jinja2 import Template

username = "Alain"

template_text = """
---
license: apache-2.0
tags:
- merge
- mergekit
- lazymergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a merge of the following models using [mergekit](https://github.com/cg123/mergekit):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

## 🧩 Configuration

```yaml
{{- yaml_config -}}
```
"""

# Create a Jinja template object
jinja_template = Template(template_text.strip())

# Get list of models from config
data = yaml.safe_load(yaml_config)
if "models" in data:
    models = [data["models"][i]["model"] for i in range(len(data["models"])) if "parameters" in data["models"][i]]
elif "parameters" in data:
    models = [data["slices"][0]["sources"][i]["model"] for i in range(len(data["slices"][0]["sources"]))]
elif "slices" in data:
    models = [data["slices"][i]["sources"][0]["model"] for i in range(len(data["slices"]))]
else:
    raise Exception("No models or slices found in yaml config")

# Fill the template
content = jinja_template.render(
    model_name=MODEL_NAME,
    models=models,
    yaml_config=yaml_config,
    username=username,
)

# Save the model card
card = ModelCard(content)
card.save('merge/README.md')

In [ ]:
from google.colab import userdata
from huggingface_hub import HfApi

username = "AlainTa"

# Defined in the secrets tab in Google Colab
api = HfApi(token=userdata.get("HuggingFace"))

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merge",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model-00001-of-00015.safetensors:   0%|          | 0.00/961M [00:00<?, ?B/s]

model-00003-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00002-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00004-of-00015.safetensors:   0%|          | 0.00/998M [00:00<?, ?B/s]

...

model-00013-of-00015.safetensors:   0%|          | 0.00/948M [00:00<?, ?B/s]

model-00014-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/755M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AlainTa/Marcoro14-7B-slerp/commit/49fc7cb27f3b7f9be5c3adf48b70751a95aeb904', commit_message='Upload folder using huggingface_hub', commit_description='', oid='49fc7cb27f3b7f9be5c3adf48b70751a95aeb904', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AlainTa/Marcoro14-7B-slerp', endpoint='https://huggingface.co', repo_type='model', repo_id='AlainTa/Marcoro14-7B-slerp'), pr_revision=None, pr_num=None)

# Inférence du modèle "mlabonne/Marcoro14–7B-slerp"


In [ ]:
!pip install -qU transformers accelerate

from transformers import AutoTokenizer
import transformers
import torch

model = "mlabonne/Marcoro14-7B-slerp"
messages = [{"role": "user", "content": "What is a large language model?"}]

tokenizer = AutoTokenizer.from_pretrained(model)

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    chat_template = f"{messages}"
)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 68.9 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/22.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.86G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.62G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [ ]:
print(outputs[0]['generated_text'])

[{'role': 'user', 'content': 'What is a large language model?'}]

A large language model is a type of artificial intelligence (AI) system that uses deep learning algorithms to analyze and generate human-like text. It is trained on vast amounts of text data, such as books, articles, and websites, to learn the patterns and relationships in the language. The model can then be used to generate new text, complete sentences, answer questions, or perform other natural language tasks.

Some well-known large language models include OpenAI's GPT-3, Google's BERT, and Facebook's RoBERTa. These models have been trained on trillions of words and can generate high-quality text that is difficult to distinguish from human writing. They have a wide range of applications, such as creating chatbots, generating content for websites or social media, and aiding in language translation.


# Inférence du modèle "AlainTa/Marcoro14–7B-slerp"

In [ ]:
!pip install -qU transformers accelerate

from transformers import AutoTokenizer
import transformers
import torch

model = "AlainTa/Marcoro14-7B-slerp"
messages = [{"role": "user", "content": "I am having a troubled relationship.  Can you give me advice?"}]

tokenizer = AutoTokenizer.from_pretrained(model)

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    chat_template = f"{messages}"
)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [ ]:
print(outputs[0]['generated_text'])

[{'role': 'user', 'content': 'I am having a troubled relationship.  Can you give me advice?'}]

As a language model, I cannot provide personal advice, but I can offer some general guidance on maintaining a healthy relationship.  Remember that relationships are complex and unique to each individual, so these tips may not apply to every situation.

1. Communication: Open and honest communication is crucial in any relationship. Share your thoughts, feelings, and concerns with your partner. Listen actively and try to understand their perspective.

2. Trust: Trust is the foundation of any healthy relationship. Build trust by being reliable, honest, and consistent. Be supportive of your partner's goals and aspirations.

3. Empathy: Put yourself in your partner's shoes and try to understand their feelings. Empathy can help you resolve conflicts and strengthen your bond.

4. Respect: Treat your partner with respect, even during difficult times. Show appreciation for their efforts and achieveme

In [ ]:
messages = [{"role": "user", "content": "Why is there sometimes a green flash at sunset and sunrise"}]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    chat_template = f"{messages}"
)



outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
print(outputs[0]['generated_text'])

[{'role': 'user', 'content': 'Why is there sometimes a green flash at sunset and sunrise'}]

[{'role': 'assistant', 'content': 'The green flash phenomenon occurs when the Sun is very close to the horizon during sunrise or sunset. As the Sun's light passes through Earth's atmosphere, the light is refracted, or bent, causing the green light to be separated from the other colors. This green light is then visible for a brief moment before or after the Sun disappears below the horizon. The green flash is often a faint and fleeting sight, making it a sought-after phenomenon by photographers and astronomy enthusiasts.'}]
